In [186]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow import keras
import numpy as np
from os import listdir
from os.path import isfile
import math

In [187]:
class SequenceGenerator(keras.utils.Sequence):
    """
    A keras Sequence to be used as an image generator for the model.
    """

    def __init__(self, x, y, batchsize):
        self.x, self.y, self.batchsize = x, y, batchsize

    def __len__(self):
        return math.ceil(len(self.x) / self.batchsize)

    def names_at_batch(self, idx):
        x_names = self.x[idx * self.batchsize:(idx + 1) * self.batchsize]
        y_names = np.asarray(self.y[idx * self.batchsize:(idx + 1) * self.batchsize])
        return x_names, y_names

    def __getitem__(self, idx):
        x_names = self.x[idx * self.batchsize:(idx + 1) * self.batchsize]
        y_names = np.asarray(self.y[idx * self.batchsize:(idx + 1) * self.batchsize])
        
        # open x image names, resize, normalise and make a numpy array
        x1 = np.asarray([preprocess_input(img_to_array(load_img(file_name[0], target_size=(299, 299)))) for file_name in x_names])
        x2 = np.asarray([preprocess_input(img_to_array(load_img(file_name[1], target_size=(299, 299)))) for file_name in x_names])
        x3 = np.asarray([preprocess_input(img_to_array(load_img(file_name[2], target_size=(299, 299)))) for file_name in x_names])

        return [x1, x2, x3], y_names

    def num_classes(self):
        ret = []
        for cat in self.y:
            if cat not in ret:
                ret.append(cat)
        return len(ret)

In [188]:
def get_filenames_from_dir(directory):
    x = []
    y = []
    category_count = 0
    for category in listdir(directory):
        triplet = []
        if isfile(category):
            continue
        for file in listdir("{0}/{1}".format(directory, category)):
            if file[-3:] != "jpg":
                continue
            triplet.append(("{0}/{1}/{2}".format(directory, category, file), category_count))
            if len(triplet) == 3:
                x.append([img[0] for img in triplet])
                y.append(triplet[0][1])
                triplet = []
        category_count += 1

    return x, y

In [189]:
# makes arrays of the images and label names
x_names, y_names = get_filenames_from_dir("database")

# 15% of all the images are set aside as the test set
x_train_val, x_test, y_train_val, y_test = train_test_split(x_names, y_names, test_size=0.15, random_state=42)

# 17% of the non-test images are set aside as the validation set
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.17, random_state=42)

In [190]:
# make generators with batch size 32 for each set
train_gen = SequenceGenerator(x_train, y_train, 1)
val_gen = SequenceGenerator(x_val, y_val, 1)
test_gen = SequenceGenerator(x_test, y_test, 1)

In [ ]:
# Model 1
base_model1 = InceptionV3(weights="imagenet")
for layer in base_model1.layers:
    layer._name += "_1"
    layer.trainable = True

# Model 2
base_model2 = InceptionV3(weights="imagenet")
for layer in base_model2.layers:
    layer._name += "_2"
    layer.trainable = True

# Model 3
base_model3 = InceptionV3(weights="imagenet")
for layer in base_model3.layers:
    layer._name += "_3"
    layer.trainable = True

In [ ]:
out_layer = keras.layers.add([base_model1.layers[-2].output, base_model2.layers[-2].output, base_model3.layers[-2].output])
predictions = keras.layers.Dense(291, activation='softmax')(out_layer)

model = keras.Model(inputs=[base_model1.input, base_model2.input, base_model3.input], outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_41_1 (InputLayer)         [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
input_42_2 (InputLayer)         [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
input_43_3 (InputLayer)         [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_5409_1 (Conv2D)          (None, 149, 149, 32) 864         input_41_1[0][0]                 
___________________________________________________________________________________________

In [ ]:
model.fit(train_gen, validation_data=val_gen, epochs=5)

Epoch 1/5
 2967/14833 [=====>........................] - ETA: 34:20 - loss: 5.8576 - accuracy: 0.0135

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_gen)

298/298 [==============================] - 93s 311ms/step - loss: 0.9547 - accuracy: 0.7321


[0.9546959400177002, 0.7321409583091736]